# 모델 불러오기 및 저장

### 4-1. 필수 라이브러리 설치

In [3]:
try:
    import google.colab
    inColab = True
except ImportError:
    inColab = False

In [ ]:
if inColab == True:
    !pip install -U pandas==2.2.2 numpy==2.0.2 scipy==1.14.1 accelerate==1.6.0 peft==0.15.2 bitsandbytes==0.45.5 transformers==4.51.3 trl==0.16.1 datasets==3.5.0 tensorboard==2.19.0

In [1]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.9 MB/s eta 0:00:00


In [7]:
if inColab == True:
    from google.colab import drive
    drive.mount("/content/gdrive")

Mounted at /content/gdrive


### 4-2. 모델 및 토크나이저 불러오기

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch

# 1. 8bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)

# 2. 모델 로딩
model = AutoModelForCausalLM.from_pretrained(
    "학습한 모델",
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained("학습한 모델")

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/57.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

### 3. 텍스트 생성 파이프라인 정의

In [6]:
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer
)

Device set to use cuda:0


### 응답 생성

In [45]:
def get_medical_response(symptom_text,num_responses=3):
    system_prompt = (
        "당신은 의료 인공지능입니다. 사용자의 증상에 따라 아래 형식으로 한 문장만 출력하세요.\n"
        "반드시 다음 형식을 지키세요:\n"
        "해당 증상으로 보아 [질환명]으로 의심됩니다. [진료과]에서 진료를 권합니다.\n"
        "그 외의 문장이나 설명은 절대 하지 마세요.\n"
        "### 증상:\n"
    )

    prompt = system_prompt + symptom_text.strip() + "\n### 답변:\n"

    outputs = pipe(
        prompt,
        max_new_tokens=90,  # 두 문장 정도가 나올 수 있도록 토큰 길이 조정
        do_sample=True,
        temperature=0.3,
        top_p=0.9,
        num_return_sequences=num_responses,
        eos_token_id=tokenizer.eos_token_id,
    )

    full_output = outputs[0]["generated_text"]
    response = full_output[len(prompt):].strip()

    # 마침표 기준으로 두 문장까지 출력
    if "." in response:
        sentences = response.split(".")
        if len(sentences) > 1:
            response = sentences[0] + "." + sentences[1] + "."  # 첫 두 문장만 출력
        else:
            response = sentences[0] + "."

    return response

In [46]:
symptom = "무릎 통증과 두통이 있습니다. 특히 앉았다 일어설 때 무릎이 아프고, 두통은 하루 종일 지속됩니다."
print(get_medical_response(symptom))

해당 증상으로 보아 근육통으로 의심됩니다. 근육통 진료를 권합니다.
